In [9]:
email_sender: str = "emailsender@gmail.com"
email_password: str = "password123"
email_receiver: str = "emailreceiver@mail.com"

In [10]:
def send_mail(subject: str, body: str, email_sender:str=email_sender, email_password:str=email_password, email_receiver:str=email_receiver):
    """
    Send an email using Gmail's SMTP server.

    Parameters:
    - subject (str): The subject of the email.
    - body (str): The body or content of the email.
    - email_sender (str): The email address of the sender (defaults to a pre-defined value).
    - email_password (str): The password for the email sender's account (defaults to a pre-defined value).
    - email_receiver (str): The email address of the receiver (defaults to a pre-defined value).

    This function sends an email with the specified subject and body from the sender's email address to the receiver's email address using Gmail's SMTP server. It uses SSL encryption for secure communication. If the email is sent successfully, it prints a success message. If there's an error during the process, it prints an error message.

    Note:
    - You should ensure that "Less secure apps" is turned on for the sender's Gmail account.
    - Use this function responsibly and avoid hardcoding sensitive credentials directly in your code.
    """
    from email.message import EmailMessage
    import ssl
    import smtplib

    em = EmailMessage()
    em["From"] = email_sender
    em["To"] = email_receiver
    em["Subject"] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
        try:
            smtp.login(email_sender, email_password)
            smtp.sendmail(email_sender, email_receiver, em.as_string())
            print("Email sent succesfuly")

        except Exception as e:
            print(f"Error: {e}")        

In [ ]:
from ultralytics import YOLO
import cv2
from collections import defaultdict
import numpy as np
import time

# load yolov8 model
model = YOLO('yolov8n.pt')

# cap = cv2.VideoCapture(0) # use your own cam
cap = cv2.VideoCapture("test8.mp4") # use saved video

# saving record as .mp4
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter('videoRecord.mp4', fourcc, 30.0, (frame_width, frame_height))

# we use it to store history of tracked objects
track_history = defaultdict(lambda: [])

# store id's for checking later
object_id_list = []

while cap.isOpened():
    # reading frames
    ret, frame = cap.read() 
    ret, frame2 = cap.read()

    # if succesful
    if ret:
        try:
            results = model.track(frame, persist=True) # track objects
            boxes = results[0].boxes.xywh.cpu() # get tracked objects' boundary boxes
            
            track_ids = results[0].boxes.id.int().cpu().tolist() # get boundary boxes' ids
            object_id_list.append(track_ids) # add to the list
        except Exception as e:
            print(f"Error while tracking (frame 1): {e}")
            continue

        try:
            results2 = model.track(frame2, persist=True)
            boxes2 = results2[0].boxes.xywh.cpu()
            track_ids_2 = results2[0].boxes.id.int().cpu().tolist()
            object_id_list.append(track_ids_2)
        except Exception as e:
            print(f"Error while tracking (frame 2): {e}")
            continue

        for id, box in zip(track_ids, boxes):
            if id not in track_ids_2:
                current_time = time.strftime("%H:%M:%S, %d-%m-%Y") # get current time

                # send email if one of the objects disappear
                send_mail(subject=f"Object {id} disappeared at {current_time}", 
                          body=f"Object {id} disappeared at {current_time}!\nYou may want to call 911!")
                
                print(f"Object {id} disappeared at {current_time}")
        
        for id, box in zip(track_ids_2, boxes):
           if id not in object_id_list:
                current_time = time.strftime("%H:%M:%S, %d-%m-%Y") # get current time

                # send email if new objects enter
                send_mail(subject=f"Object {id} entered at {current_time}", 
                          body=f"Object {id} entered at {current_time}!\nYou may want to call 911!\nSaved a photo of the object.")
                cv2.imwrite(f'id_{id}_entered.png', frame)
                object_id_list.append(id)
           
        frame_ = results[0].plot()
                
        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            
            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame_, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # write to an mp4 file
        out.write(frame_)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

        cv2.imshow('frame', frame_)

cap.release()
out.release()
cv2.destroyAllWindows()